#### 1. 누락 경로 필터링 (LibriSpeech/train-other-500)

#### 2. stage-1
- task별로 분리해준 후에 
- 오디오 길이 기준 이상치(상-하위 5%)를 제거하고
- 다시 텍스트 기준 10분위 분할한 후에
- 각 분위에서 설정한 갯수에 맞춰 샘플링

#### 3. stage-2
- task별로 분리해준 후에 
- 오디오 길이 기준 이상치(상-하위 5%)를 제거하고
- 다시 텍스트 기준 10분위 분할한 후에
- 각 분위에서 설정한 갯수에 맞춰 샘플링
- filtered_stage1_train.json에 대하여 asr, audiocaption task 각각에 대해
오디오 길이 기준 상위 하위 5퍼센트를 제외하고, 중복을 피해 각 1만개씩만 stage-2로 추가 샘플링

#### 4. 중복경로 제거


In [2]:
# 1. 누락경로 제거

import json
import os

def filter_librispeech_other500(input_files):
    for file_path in input_files:
        # 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 처리 전 개수 저장
        before_count = len(data['annotation'])
        
        # LibriSpeech/train-other-500으로 시작하는 path를 가진 항목 필터링
        filtered_annotations = [
            item for item in data['annotation'] 
            if not item['path'].startswith('/LibriSpeech/train-other-500')
        ]
        
        # 필터링된 결과로 데이터 업데이트
        data['annotation'] = filtered_annotations
        
        # 처리 후 개수 계산
        after_count = len(data['annotation'])
        
        # 결과 출력
        print(f"File: {file_path}")
        print(f"Before filtering: {before_count} items")
        print(f"After filtering: {after_count} items")
        print(f"Removed: {before_count - after_count} items\n")
        
        # 현재 폴더에 새 파일명으로 저장
        original_filename = os.path.basename(file_path)
        new_filename = f"filtered_{original_filename}"
        
        # 한 줄로 저장 (공백 없이)
        with open(new_filename, 'w', encoding='utf-8') as f:
            f.write(json.dumps(data, ensure_ascii=False, separators=(', ', ': ')))
        
        print(f"Saved as: {new_filename}\n")

# 파일 경로 리스트
input_files = ['stage1_train.json', 'stage2_train.json']

# 실행
filter_librispeech_other500(input_files)

File: stage1_train.json
Before filtering: 1573558 items
After filtering: 1424870 items
Removed: 148688 items

Saved as: filtered_stage1_train.json

File: stage2_train.json
Before filtering: 1403890 items
After filtering: 957922 items
Removed: 445968 items

Saved as: filtered_stage2_train.json



In [4]:
# 2. stage-1 생성

import json
import os
import numpy as np
import soundfile as sf
from collections import defaultdict

# 기본 설정
JSON_DIR = "/data/hsk/data_check"  # JSON이 있는 디렉토리 
FILE_DIR = "/data/data_storage"  # 오디오 데이터 폴더가 있는 디렉토리
INPUT_FILE = os.path.join(JSON_DIR, "filtered_stage1_train.json") # 이전 단계에서 생성된 데이터 파일 이름
OUTPUT_FILE = "final_stage_1.json"
TASK_SIZES = {"asr": 6000, "audiocaption": 4000}

# JSON 데이터 로드
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

data = load_json(INPUT_FILE)

# Task별로 데이터 분리
task_data = defaultdict(list)
for item in data["annotation"]:
    task_data[item["task"].lower()].append(item)

# 오디오 길이 계산 함수
def get_audio_duration(audio_path):
    full_path = os.path.join(FILE_DIR, audio_path.lstrip("/"))
    try:
        with sf.SoundFile(full_path) as f:
            return len(f) / f.samplerate
    except Exception as e:
        print(f"Error processing {full_path}: {e}")
        return None

# 오디오 길이 상위/하위 5% 제거
def filter_by_duration(task_items):
    durations = [get_audio_duration(item["path"]) for item in task_items]
    durations = np.array([d for d in durations if d is not None])
    
    if len(durations) == 0:
        return []
    
    lower, upper = np.percentile(durations, [5, 95])
    return [item for item, d in zip(task_items, durations) if lower <= d <= upper]

# 텍스트 길이 기준 10분위 분할
def split_by_text_length(task_items):
    task_items.sort(key=lambda x: len(x["text"]))
    split_data = np.array_split(task_items, 10)
    return [list(split) for split in split_data]

# 각 task별 처리 및 샘플링
final_data = []
for task, items in task_data.items():
    filtered_items = filter_by_duration(items)
    split_items = split_by_text_length(filtered_items)
    
    if task in TASK_SIZES:
        sample_size = TASK_SIZES[task]
        for split in split_items:
            final_data.extend(split[:sample_size])

# 최종 JSON 저장
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump({"annotation": final_data}, f, indent=4, ensure_ascii=False)

print(f"Final dataset saved to {OUTPUT_FILE}")


# 이하 세개 오디오 파일이 정상적이지 않으므로 해당 에러는 무시
# GigaSpeech/26/YOU0000008873_S0000015.wav
# GigaSpeech/21/YOU0000003270_S0000046.wav
# GigaSpeech/26/YOU0000008877_S0000500.wav

Error processing /data/data_storage/GigaSpeech/26/YOU0000008873_S0000015.wav: Error opening '/data/data_storage/GigaSpeech/26/YOU0000008873_S0000015.wav': System error.
Error processing /data/data_storage/GigaSpeech/21/YOU0000003270_S0000046.wav: Error opening '/data/data_storage/GigaSpeech/21/YOU0000003270_S0000046.wav': System error.
Error processing /data/data_storage/GigaSpeech/26/YOU0000008877_S0000500.wav: Error opening '/data/data_storage/GigaSpeech/26/YOU0000008877_S0000500.wav': System error.
Final dataset saved to final_stage_1.json


In [5]:
# 3. stage-2 생성

import json
import os
import numpy as np
import soundfile as sf
from collections import defaultdict

# 기본 설정
ROOT_DIR = "/data/data_storage"
INPUT_FILES = [
    os.path.join(ROOT_DIR, "filtered_stage1_train.json"),
    os.path.join(ROOT_DIR, "filtered_stage2_train.json"),
]
OUTPUT_FILE = "final_stage_2.json"
TASK_SIZES_STAGE2 = {
    "asr": 3000,
    "audiocaption_v2": 3200,
    "phone_recognition": 2000,
    "qa": 4500,
    "gender_recognition": 1400,
}
TASK_SIZES_STAGE1 = {"asr": 20000, "audiocaption": 20000}

# JSON 데이터 로드
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

data_stage1 = load_json(INPUT_FILES[0])
data_stage2 = load_json(INPUT_FILES[1])

task_data = defaultdict(list)
for item in data_stage2["annotation"]:
    task_data[item["task"].lower()].append(item)

# 오디오 길이 계산 함수
def get_audio_duration(audio_path):
    full_path = os.path.join(ROOT_DIR, audio_path.lstrip("/"))
    try:
        with sf.SoundFile(full_path) as f:
            return len(f) / f.samplerate
    except Exception as e:
        print(f"Error processing {full_path}: {e}")
        return None

# 오디오 길이 상위/하위 5% 제거
def filter_by_duration(task_items):
    durations = [get_audio_duration(item["path"]) for item in task_items]
    durations = np.array([d for d in durations if d is not None])
    
    if len(durations) == 0:
        return []
    
    lower, upper = np.percentile(durations, [5, 95])
    return [item for item, d in zip(task_items, durations) if lower <= d <= upper]

# 텍스트 길이 기준 10분위 분할
def split_by_text_length(task_items):
    task_items.sort(key=lambda x: len(x["text"]))
    split_data = np.array_split(task_items, 10)
    return [list(split) for split in split_data]

# 각 task별 처리 및 샘플링
final_data = []
for task, items in task_data.items():
    filtered_items = filter_by_duration(items)
    split_items = split_by_text_length(filtered_items)
    
    if task in TASK_SIZES_STAGE2:
        sample_size = TASK_SIZES_STAGE2[task]
        for split in split_items:
            final_data.extend(split[:sample_size])

# Stage1에서 asr, audiocaption 추가
task_data_stage1 = defaultdict(list)
for item in data_stage1["annotation"]:
    if item["task"].lower() in TASK_SIZES_STAGE1:
        task_data_stage1[item["task"].lower()].append(item)

for task, items in task_data_stage1.items():
    filtered_items = filter_by_duration(items)
    selected_items = filtered_items[:TASK_SIZES_STAGE1[task]]
    final_data.extend(selected_items)

# 최종 JSON 저장
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump({"annotation": final_data}, f, indent=4, ensure_ascii=False)

print(f"Final dataset saved to {OUTPUT_FILE}")


# 이하 세개 오디오 파일이 정상적이지 않으므로 해당 에러는 무시
# GigaSpeech/26/YOU0000008873_S0000015.wav
# GigaSpeech/21/YOU0000003270_S0000046.wav
# GigaSpeech/26/YOU0000008877_S0000500.wav

Error processing /data/data_storage/GigaSpeech/26/YOU0000008873_S0000015.wav: Error opening '/data/data_storage/GigaSpeech/26/YOU0000008873_S0000015.wav': System error.
Error processing /data/data_storage/GigaSpeech/21/YOU0000003270_S0000046.wav: Error opening '/data/data_storage/GigaSpeech/21/YOU0000003270_S0000046.wav': System error.
Error processing /data/data_storage/GigaSpeech/26/YOU0000008877_S0000500.wav: Error opening '/data/data_storage/GigaSpeech/26/YOU0000008877_S0000500.wav': System error.
Final dataset saved to final_stage_2.json


In [7]:
# 4-1 stage-1 중복 제거

import json
from collections import Counter

def process_data(file_path):
    # 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 중복 제거 전 task 개수 계산
    before_counts = Counter(item['task'] for item in data['annotation'])
    print("\n중복 제거 전 task 별 개수:")
    for task, count in before_counts.items():
        print(f"{task}: {count}")
    print(f"총 개수: {len(data['annotation'])}")
    
    # path를 기준으로 중복 제거
    seen_paths = set()
    unique_annotations = []
    
    for item in data['annotation']:
        if item['path'] not in seen_paths:
            seen_paths.add(item['path'])
            unique_annotations.append(item)
    
    # 중복 제거 후 task 개수 계산
    after_counts = Counter(item['task'] for item in unique_annotations)
    print("\n중복 제거 후 task 별 개수:")
    for task, count in after_counts.items():
        print(f"{task}: {count}")
    print(f"총 개수: {len(unique_annotations)}")
    
    # 제거된 중복 개수 계산
    duplicates_removed = len(data['annotation']) - len(unique_annotations)
    print(f"\n제거된 중복 항목 수: {duplicates_removed}")
    
    # 결과를 새 파일로 저장
    output_data = {'annotation': unique_annotations}
    output_path = file_path.replace('.json', '_no_duplicates.json')
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=4)
    
    print(f"\n중복이 제거된 데이터가 저장됨: {output_path}")

# 파일 처리 실행
file_path = 'final_stage_1.json'
process_data(file_path)


중복 제거 전 task 별 개수:
asr: 60000
audiocaption: 40000
총 개수: 100000

중복 제거 후 task 별 개수:
asr: 60000
audiocaption: 39731
총 개수: 99731

제거된 중복 항목 수: 269

중복이 제거된 데이터가 저장됨: final_stage_1_no_duplicates.json


In [8]:
# 4-2 stage-2 중복 제거

import json
from collections import Counter

def process_data(file_path):
    # 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 중복 제거 전 task 개수 계산
    before_counts = Counter(item['task'] for item in data['annotation'])
    print("\n중복 제거 전 task 별 개수:")
    for task, count in before_counts.items():
        print(f"{task}: {count}")
    print(f"총 개수: {len(data['annotation'])}")
    
    # path를 기준으로 중복 제거
    seen_paths = set()
    unique_annotations = []
    
    for item in data['annotation']:
        if item['path'] not in seen_paths:
            seen_paths.add(item['path'])
            unique_annotations.append(item)
    
    # 중복 제거 후 task 개수 계산
    after_counts = Counter(item['task'] for item in unique_annotations)
    print("\n중복 제거 후 task 별 개수:")
    for task, count in after_counts.items():
        print(f"{task}: {count}")
    print(f"총 개수: {len(unique_annotations)}")
    
    # 제거된 중복 개수 계산
    duplicates_removed = len(data['annotation']) - len(unique_annotations)
    print(f"\n제거된 중복 항목 수: {duplicates_removed}")
    
    # 결과를 새 파일로 저장
    output_data = {'annotation': unique_annotations}
    output_path = file_path.replace('.json', '_no_duplicates.json')
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=4)
    
    print(f"\n중복이 제거된 데이터가 저장됨: {output_path}")

# 파일 처리 실행
file_path = 'final_stage_2.json'
process_data(file_path)


중복 제거 전 task 별 개수:
asr: 50000
QA: 45000
phone_recognition: 20000
audiocaption_v2: 32000
gender_recognition: 14000
audiocaption: 20000
총 개수: 181000

중복 제거 후 task 별 개수:
asr: 37098
QA: 43762
phone_recognition: 15871
audiocaption_v2: 26979
gender_recognition: 9435
audiocaption: 17444
총 개수: 150589

제거된 중복 항목 수: 30411

중복이 제거된 데이터가 저장됨: final_stage_2_no_duplicates.json
